In [1]:
import datetime as dt
import warnings
import numpy as np
from utils import database_table
warnings.filterwarnings("ignore")

In [2]:
pp_projs = database_table("prizepicks_projections")

model_predicted_series = np.where(
(pp_projs["line_score"] < pp_projs["predict"]),
"Over",
"Under"   
)

outcome_series = np.where(
    ((pp_projs["predict"] > pp_projs["line_score"]) & (pp_projs["actual"] > pp_projs["line_score"])) |
    ((pp_projs["predict"] < pp_projs["line_score"]) & (pp_projs["actual"] < pp_projs["line_score"])),
    "Cash",
    "Chalk"
)

pp_projs["model_predicted"] = model_predicted_series
pp_projs["outcome"] = outcome_series
pp_projs.head()

[STATUS]: SUCCESSFULLY CREATED DB CONNECTION


,id,game_date,stat_type,player_name,player_team,opp,line_score,player_id,team_id,predict,actual,model_predicted,outcome
0,3245390,2024-10-30,MAPS 3 Kills,Chr1zn,OG,9INE,14.5,206557.0,23968.0,15.8832,15.0,Over,Cash
1,3245391,2024-10-30,MAPS 3 Kills,F1KU,OG,9INE,15.5,169109.0,23968.0,14.1549,10.0,Under,Cash
2,3245392,2024-10-30,MAPS 3 Kills,MoDo,OG,9INE,14.0,168562.0,23968.0,14.1897,20.0,Over,Cash
3,3245393,2024-10-30,MAPS 3 Kills,Buzz,OG,9INE,15.5,206559.0,23968.0,14.8314,14.0,Under,Cash
4,3245394,2024-10-30,MAPS 3 Kills,M1key,OG,9INE,15.0,221874.0,23968.0,13.4820,13.0,Under,Cash


In [3]:
def results_summary(days_num: int):
    specific_date = dt.date.today() - dt.timedelta(days=days_num)
    pp_projs_subset = pp_projs[specific_date <= pp_projs["game_date"]]
    overall_hr = (pp_projs_subset["outcome"] == "Cash").sum()
    map_report = pp_projs_subset.groupby("stat_type").apply(lambda group: group["outcome"].value_counts(normalize=True)*100)
    over_under_report = pp_projs_subset.groupby("model_predicted").apply(lambda group: group["outcome"].value_counts(normalize=True)*100)
    print(f"SUMMARY REPORT AFTER {specific_date}")
    print(f"Overall HR: {round(overall_hr/len(pp_projs_subset), 2)*100}% ({overall_hr}/{len(pp_projs_subset)})")
    print()
    print("MAPS CASHED")
    print("-"*50)
    for map_type in map_report.index:
        try:
            print(f"{map_type}: {round(map_report.loc[map_type, 'Cash'], 2)}%")
        except KeyError:
            continue
    print()
    print("O/U CASHED")
    print("-"*50)
    for over_or_under in ["Over", "Under"]:
        try:
            print(f"{over_or_under}: {round(over_under_report.loc[over_or_under, 'Cash'], 2)}%")
        except KeyError:
            continue

In [4]:
input_days = int(input("NUMBER OF DAYS: "))
results_summary(input_days)

SUMMARY REPORT AFTER 2025-02-19
Overall HR: 52.0% (1384/2654)

MAPS CASHED
--------------------------------------------------
MAPS 1-2 Headshots: 52.82%
MAPS 1-2 Kills: 52.51%
MAPS 3 Headshots: 50.57%
MAPS 3 Kills: 50.85%

O/U CASHED
--------------------------------------------------
Over: 51.43%
Under: 53.24%
